In [ ]:
import warnings
import graphviz
import re
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.tools.eval_measures import rmse
from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공선성 패키지 불러오기
from sklearn.linear_model import LinearRegression # sklearn 선형회귀 모형 -> 변수 선택법을 위함
from sklearn.feature_selection import RFE # 변수 선택법 (후진 제거법)
from sklearn.preprocessing import scale # 데이터 scale
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, fbeta_score
# Import ML Classification algos
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')
%matplotlib inline
matplotlib.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus']=False
data = pd.read_csv("./유방암.CSV",engine="python")
data.head()

# 데이터 결측치 확인

In [ ]:
# 결측치 없다.
data.isnull().sum(axis=0)

# 데이터 타입 확인

In [ ]:
data.info()

In [ ]:
# Checking if classes are unbalanced or balanced
data['diagnosis'].value_counts()

# 'diagonosis'는 목표변수이다.
# 0은 악성이다. 1은 양성이다.
- '1' 은 Malignant으로 악성이다 -> 212/(212 + 357) = 0.37 
- '0' 은 Benign으로 양성이다 -> 357/(212+357) = 0.627

In [ ]:
sns.countplot(data.diagnosis)
sns.countplot(data['diagnosis'],  palette = "husl")

# Correlation 비교
- 목표변수와 설명변수간의 상관계수 비교
- 상관관계를 확인해 크게 연관이 있는지 파악한다
- 상관관계는 인과관계를 내포하지 않아 상관관계가 큰 특징들이 목표변수에 무조건 변화를 일으키는 것은 아니다.
- **중요한 것은 음의 상관관계를 갖는 특성도 모델을 사용해 실제로 상관관계가 있는지 파악해야한다.**

In [ ]:
# Check 설명변수와 목표변수간의 상관계수  
print('Features with the max correlation with diagnosis :')
# sort를 사용해 큰 순서로 상관계수 10개 뽑기
print(data.corr()['diagnosis'][1:-1].sort_values(ascending=False)[:10])
print('\n')
# sort를 사용해 작은 순서로 상관계수 10개 뽑기
print('Features with the least correlation with diagnosis :')
print(data.corr()['diagnosis'][:-1].sort_values()[:10]) 

# No insights
- 다른 특징들의 대부분에 차이가 있음을 알 수 있다.
- **다양한 특징의 범위가 다르게 보이기 때문에 normalize/scale이 필요하다**

In [ ]:
X_log.hist(figsize = (20,20))

# 데이터 스케일
- **1. 모든 분포가 왼쪽으로 치우쳐 있기 때문에 로그 변환을 적용해 분포를 보다 정상으로 만들고 왼쪽 꼬리를 중앙으로 sacle한다**

- **2. 전체 데이터를 (0,1)사이의 MinMaxScaler과 평균 0 표준 편차 1 만들어 값을 변화시킨다.**

In [ ]:
y = data.loc[:,'diagnosis']
X = data.iloc[:,2:-1]

X_log = X.apply(lambda x: np.log(x + 0.1))

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_log)
X_log.hist(figsize = (20,20))

In [ ]:
# 데이터 분할(train, test 데이터 7:3으로 - numpy->random)
np.random.seed(seed=1234)

# 0.7보다 작으면 True -> train 데이터, 아니면 False -> test 데이터
msk = np.random.rand(data.shape[0]) < 0.7
data_train = data[msk]
data_test = data[~msk]

# train/test 데이터의 목표변수/설명변수 지정
data_train_y = data_train["diagnosis"]
data_train_x = data_train.drop("diagnosis", axis=1, inplace=False)
data_test_y = data_test["diagnosis"]
data_test_x = data_test.drop("diagnosis", axis=1, inplace=False)

# train데이터와 test데이터 크기
print("train data X size: {}".format(data_train_x.shape))
print("train data Y size: {}".format(data_train_y.shape))
print("test data X size: {}".format(data_test_x.shape))
print("test data Y size: {}".format(data_test_y.shape))
# train data: 385, test data: 184

# 의사결정나무 (Decision Tree)

In [ ]:
tree_uncustomized = DecisionTreeClassifier(random_state=1234)
tree_uncustomized.fit(data_train_x, data_train_y)

# 훈련 데이터 정확도
print("Accuracy on training set:{:.3f}".format(tree_uncustomized.score(data_train_x,
                                                                      data_train_y)))
print("Accuracy on training set:{:3f}".format(tree_uncustomized.score(data_test_x,
                                                                      data_test_y)))


# tree model 옵션 확인

In [ ]:
tree_uncustomized.get_params

# GridSearchCV 사용
- **CRiterion: 분리 기준 변경(gini, entropy)**
- **depth는 1~50까지 잡는다. (tree - 최대 깊이)**
- **num_leafs는 Tree의 leaf**

In [ ]:
depths = np.arange(1,51)
num_leafs = [1,5,10,20,50,100]
para_criterion = ["gini", "entropy"]
param_grid = [{'criterion' : para_criterion, 'max_depth': depths, 'min_samples_leaf':num_leafs}]

tree_GS = DecisionTreeClassifier(random_state=1234)
gs = GridSearchCV(estimator=tree_GS, param_grid=param_grid,cv=10)
gs = gs.fit(data_train_x,data_train_y)
gs

# GridSearch 출력
- **GridSearch를 했을 때 가장좋은 test accuracy중 가장 좋은 score를 출력**
- **그 때 조합들의 parameter를 출력했다.**

In [ ]:
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
my_model = gs.best_estimator_
my_model.fit(data_train_x,data_train_y)

# GridSearch의 Parameter로 DecisionTree 출력

In [ ]:
tree = DecisionTreeClassifier(criterion="gini",max_depth=5,min_samples_leaf=1, random_state=1234)
tree.fit(data_train_x, data_train_y)
export_graphviz(tree, out_file="tree.dot", class_names=["0","1"],
               feature_names=data_train_x.columns, impurity=False, filled=True)

os.environ["PATH"] += os.pathsep + "C:/Program Files (x86)/Graphviz2.38/bin"

with open("tree.dot") as f:
    dot_graph = f.read()
display(graphviz.Source(dot_graph))

In [ ]:
# 최종 모델
rf_model = DecisionTreeClassifier(criterion="gini",max_depth=5,min_samples_leaf=1,random_state=1234)
rf_model.fit(data_train_x,data_train_y)
# rf_model.featrue_importance_로 설명변수 중요도 확인
ds_feature_importance = pd.DataFrame()
ds_feature_importance["feature"] = data_train_x.columns
ds_feature_importance["importance"] = rf_model.feature_importances_
ds_feature_importance.sort_values(by="importance",ascending=False)


In [ ]:
def plot_feature_importances(model):
    n_feature = data_train_x.shape[1]
    plt.barh(range(n_feature),model.feature_importances_,align="center")
    plt.yticks(np.arange(n_feature), data_train_x.columns)
    plt.xlabel("설명변수 중요도")
    plt.ylabel("설명변수")
    plt.ylim(-1, n_feature)
# 설명변수 중요도 그리는 함수 실행
plot_feature_importances(rf_model)
    